# Euclid Data Collection  

Scrape from Reddit as many posts as possible that pertain to the Euclic galaxy.

In [2]:
import random
import my_util as u
import pandas as pd
from math import floor
from my_util import gallery_first_image, url_to_img, post_info, nmsce
from itertools import product
from datetime import datetime
import tensorflow as tf

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [3]:
import praw
import numpy as np
from math import ceil
import urllib.request
from PIL import Image

reddit = praw.Reddit( \
    client_id='VZDFQR1wT8giqAVcq09UmQ', \
    client_secret='27AnuMqvvV6CbxekJspgAyxD4UYThw', \
    user_agent='nms-screenshot-scraper', \
    username='nms_cartographer', \
    password='y1C1FMt2X9mu')

nmsce = reddit.subreddit('NMSCoordinateExchange')

def gallery_first_image(gallery_url):
    """Gets first image from a reddit gallery post
    """

    try:
        submission = reddit.submission(url = gallery_url)
        image_dict = submission.media_metadata
        # return re.sub('width=[0-9]+', 'width=1920', list(image_dict.values())[0]['s']['u'])
        return list(image_dict.values())[0]['s']['u']
    except: 
        return ""

def url_to_img(url):
    """Use pillow to create a pillow.image object from a url
    """

    ext = url.split(".")[-1].split("?")[0]
    temp = "temp." + ext
    urllib.request.urlretrieve(url, temp)
    return Image.open(temp)

class post_info():
    """ Simple class holding a subset of 'PRAW.submission' attributes 
    """

    def __init__(self, post, flair_text):
        self.post_id = post.id
        self.title = post.title
        try:
            self.author_id = post.author.id
            # self.author_name = post.author.name
        except:            
            self.author_id = "-1"
            self.author_name = "deleted"
        # self.upvotes = post.score
        self.flair_text = flair_text
        # self.flair_text = post.link_flair_text
        self.url = post.url
        self.address = "000000000000"
        self.planet_index = -1
        self.system_index = -1
        self.coord_y =-1
        self.coord_x =-1
        self.coord_z =-1
        
    def process_address(self, model):
        """Extracts string address and numeric coordinates from post image
        
        Processes image, crops address, classifies runes, extracts address string, populates coords
        """
        # Get image
        try:
            if self.url.split("/")[-2] == "gallery":
                self.url = gallery_first_image(self.url)
        except:
            self.url = ""
        # Preprocess
        # For each glyph
            # classify glyph using model
            # append character to address string
        if "u" in self.address: 
            self.address = ""
            return
        # Convert hex strings to decimal, store as numeric addresses
        self.planet_index   = int(self.address[0], 16)
        self.system_index   = int(self.address[1:4], 16)
        self.coord_y        = int(self.address[4:6], 16)
        self.coord_x        = int(self.address[6:9], 16)
        self.coord_z        = int(self.address[9:], 16)

    def to_dict(self):
        """Extracts attributes to dictionary
        """
        return {
            'post_id': self.post_id,
            # 'title': self.title,
            'author_id': self.author_id,
            # 'author_name': self.author_name,
            # 'upvotes': self.upvotes,
            'flair_text': self.flair_text,
            'subject' : self.flair_text.split("/")[0],
            'galaxy' : self.flair_text.split("/")[1],
            'update' : self.flair_text.split("/")[2],
            'url': self.url,            
            'address': self.address,
            'body_index_h': int(self.address[0], 16), # Convert from hex string to integer value
            'sys_index_h': int(self.address[1:4], 16),
            'region_y_h': int(self.address[4:6], 16),
            'region_z_h': int(self.address[6:9], 16),
            'region_x_h': int(self.address[9:], 16) 
        }



To get more posts, can add update number as a flair filter. While there are a couple dozen numbered updates, only updates since 3.5 are covered by flair searches. 

We can also scrape a full set for each subject category. Multitool posts are less common and tend to be formatted differently, so we will ignore those. We will also ignore bases because those are platform specific and usually don't make for as nice of screenshots. 

4 updates and 6 categories of interest gives us 24 loops of however many posts. Reddit caps requests at 1000, but I've only managed to get 250 at once with PRAW, and that is not quick. So I'll try with 150 per category per update. This should a few thousand data points even after removing the ones that can't be classified by the model. 

In [4]:
updates = [3.35, 3.37, 3.5, 3.6, 3.7, 3.8, 3.82, 3.88, 3.89, 3.94, 4]
subjects = ['Planet', 'Living_Ship', 'Starship', 'Fauna','Freighter','Frigate']
limit = None
period = 'all'      # Most of these updates are older than a year
galaxy = "Euclid"


In [13]:
subset_max = len(updates)*len(subjects)
subset_num = 0
start=datetime.now()
posts = []          # Start with empty list
for update, subject in product(updates, subjects):
    subset_num += 1    
    flair_text = '{}/{}/{}'.format(subject, galaxy, str(update))
    query = 'flair:{}'.format(flair_text)
    print('Searching...\t{0: <30}'.format(query), end="")
    try: 
        [posts.append(post_info(post, flair_text)) for post in nmsce.search(query=query, sort="new", time_filter=period, limit=limit)]
    except:
        pass
    print('({} of {})'.format(subset_num, subset_max), end="\t")
    print('Posts: {}'.format(str(len(posts))), end="\t")
    elapsed = datetime.now() - start
    print('elapsed: {}'.format(str(elapsed).split(".")[0]))


Searching...	flair:/3.35/Euclid/Planet     (1 of 66)	Posts: 40	elapsed: 0:00:09
Searching...	flair:/3.37/Euclid/Planet     (2 of 66)	Posts: 124	elapsed: 0:00:30
Searching...	flair:/3.5/Euclid/Planet      (3 of 66)	Posts: 359	elapsed: 0:01:28
Searching...	flair:/3.6/Euclid/Planet      (4 of 66)	Posts: 552	elapsed: 0:04:32
Searching...	flair:3.7/Euclid/Planet       (5 of 66)	Posts: 787	elapsed: 0:08:30
Searching...	flair:/3.8/Euclid/Planet      (6 of 66)	Posts: 802	elapsed: 0:08:46
Searching...	flair:/3.82/Euclid/Planet     (7 of 66)	Posts: 877	elapsed: 0:10:02
Searching...	flair:/3.88/Euclid/Planet     (8 of 66)	Posts: 906	elapsed: 0:10:32
Searching...	flair:/3.89/Euclid/Planet     (9 of 66)	Posts: 1083	elapsed: 0:13:31
Searching...	flair:/3.94/Euclid/Planet     (10 of 66)	Posts: 1298	elapsed: 0:17:09
Searching...	flair:/4/Euclid/Planet        (11 of 66)	Posts: 1506	elapsed: 0:20:40
Searching...	flair:/3.35/Euclid/Living_Ship(12 of 66)	Posts: 1529	elapsed: 0:21:04
Searching...	flair:/3.

In [5]:
for post in posts:
    if post.address == "": 
        post.address = "000000000000" 
        print('Address Updated')


NameError: name 'posts' is not defined

### Extract Address Data from Posted Images

In [16]:
df = pd.DataFrame.from_records([post.to_dict() for post in posts])
convert_dict = {
    "address": int,
    "body_index_h": int,
    "sys_index_h": int,
    "region_y_h": int,
    "region_z_h": int,
    "region_x_h": int
}
df = df.astype(convert_dict)
df.dtypes
df

,post_id,author_id,flair_text,subject,galaxy,update,url,address,body_index_h,sys_index_h,region_y_h,region_z_h,region_x_h
0,mvgngh,5eogb,/3.35/Euclid/Planet,,3.35,Euclid,https://i.redd.it/lskdbrpqaju61.png,0,0,0,0,0,0
1,mvd34q,a0vbbj20,/3.35/Euclid/Planet,,3.35,Euclid,https://www.reddit.com/r/NMSCoordinateExchange...,0,0,0,0,0,0
2,muudaw,5eogb,/3.35/Euclid/Planet,,3.35,Euclid,https://i.redd.it/wgrwxwh5ucu61.png,0,0,0,0,0,0
3,muoikz,905lb9fe,/3.35/Euclid/Planet,,3.35,Euclid,https://i.redd.it/0v7j2wqdgbu61.jpg,0,0,0,0,0,0
4,muj1up,85ogpn3s,/3.35/Euclid/Planet,,3.35,Euclid,https://i.redd.it/achtdzj9f9u61.png,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8624,y4jijf,rksi9n4c,/4/Euclid/Frigate,,4,Euclid,https://www.reddit.com/gallery/y4jijf,0,0,0,0,0,0
8625,y48ser,4evafh,/4/Euclid/Frigate,,4,Euclid,https://www.reddit.com/gallery/y48ser,0,0,0,0,0,0
8626,y36jac,394r969d,/4/Euclid/Frigate,,4,Euclid,https://i.redd.it/0be1iapucmt91.jpg,0,0,0,0,0,0
8627,y19giz,2pc4irr8,/4/Euclid/Frigate,,4,Euclid,https://www.reddit.com/gallery/y19giz,0,0,0,0,0,0


In [27]:
df.galaxy = "Euclid"
df.subject = df.flair_text.str.split('/').str[3]
df['update'] = df.flair_text.str.split('/').str[1]
df


,post_id,author_id,flair_text,subject,galaxy,update,url,address,body_index_h,sys_index_h,region_y_h,region_z_h,region_x_h
0,mvgngh,5eogb,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/lskdbrpqaju61.png,0,0,0,0,0,0
1,mvd34q,a0vbbj20,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://www.reddit.com/r/NMSCoordinateExchange...,0,0,0,0,0,0
2,muudaw,5eogb,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/wgrwxwh5ucu61.png,0,0,0,0,0,0
3,muoikz,905lb9fe,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/0v7j2wqdgbu61.jpg,0,0,0,0,0,0
4,muj1up,85ogpn3s,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/achtdzj9f9u61.png,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8624,y4jijf,rksi9n4c,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y4jijf,0,0,0,0,0,0
8625,y48ser,4evafh,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y48ser,0,0,0,0,0,0
8626,y36jac,394r969d,/4/Euclid/Frigate,Frigate,Euclid,4,https://i.redd.it/0be1iapucmt91.jpg,0,0,0,0,0,0
8627,y19giz,2pc4irr8,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y19giz,0,0,0,0,0,0


Save Euclid dataframe as before messing with it too much. 

In [28]:
df.to_csv('euclid_data.csv')

In [9]:
df = pd.read_csv('euclid_data.csv')


In [12]:
df

,Unnamed: 0,post_id,author_id,flair_text,subject,galaxy,update,url,address,body_index_h,sys_index_h,region_y_h,region_z_h,region_x_h
0,0,mvgngh,5eogb,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/lskdbrpqaju61.png,0,0,0,0,0,0
1,1,mvd34q,a0vbbj20,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://www.reddit.com/r/NMSCoordinateExchange...,0,0,0,0,0,0
2,2,muudaw,5eogb,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/wgrwxwh5ucu61.png,0,0,0,0,0,0
3,3,muoikz,905lb9fe,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/0v7j2wqdgbu61.jpg,0,0,0,0,0,0
4,4,muj1up,85ogpn3s,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/achtdzj9f9u61.png,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8624,8624,y4jijf,rksi9n4c,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y4jijf,0,0,0,0,0,0
8625,8625,y48ser,4evafh,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y48ser,0,0,0,0,0,0
8626,8626,y36jac,394r969d,/4/Euclid/Frigate,Frigate,Euclid,4,https://i.redd.it/0be1iapucmt91.jpg,0,0,0,0,0,0
8627,8627,y19giz,2pc4irr8,/4/Euclid/Frigate,Frigate,Euclid,4,https://www.reddit.com/gallery/y19giz,0,0,0,0,0,0


Define glyph extraction function to use with dataframe

In [10]:
import urllib.request
from PIL import Image
import numpy as np

def gallery_first_image(gallery_url):
    """Gets first image from a reddit gallery post

    requires reddit object be defined 
    """

    try:
        submission = reddit.submission(url = gallery_url)
        image_dict = submission.media_metadata
        return list(image_dict.values())[0]['s']['u']
    except: 
        return ""

def url_to_img(url):
    """Use pillow to create a pillow.image object from a url
    """

    ext = url.split(".")[-1].split("?")[0]
    temp = "temp." + ext
    # print(url)
    # print(temp)
    urllib.request.urlretrieve(url, temp)
    img = Image.open(temp)
    return img

def img_to_address(screenshot):
    width, height = screenshot.size
    glyph_side = width / 60
    left = glyph_side * (1/3)
    top = height-(2*glyph_side)
    right = glyph_side*(12 + (1/3))
    bottom = height-glyph_side
    return screenshot.crop((left, top, right, bottom))

def color_to_binary(im_addr):
    """converts color address to black/white 

    dynamically attempts to adjust threshold where the first attempt is too dark or too light
    """
    threshold = 200    
    # threshold_increment = .05
    min_ratio = 15
    max_ratio = 20
    min_threshold = 128
    max_threshold = 254
    # Convert color space
    # Grayscale (probably unnecessary)
    im_addr_gray = im_addr.convert("L")   
    # Binary (definitely excessive)
    im_addr_binary= im_addr_gray.point( lambda p: 255 if p > threshold else 0 )
    hist = im_addr_binary.histogram()
    # Simple, clumsy, dynamic threshold adjustment
    try:
        ratio = hist[0]/(hist[-1]+1)
        while ratio > max_ratio or ratio < min_ratio:
            if (ratio > max_ratio): # Too Dark
                # threshold *= (1-threshold_increment)
                threshold -= 1
                # print("Too Dark", end='\t')
                # print('Threshold decreased to: {} '.format(str(ceil(threshold))), end='\t')
            else:                
                # threshold *= (1+threshold_increment)
                threshold += 1
                # print("Too Light", end='\t')
                # print('Threshold increased to: {} '.format(str(ceil(threshold))), end='\t')
            if threshold < min_threshold:
                # print("Threshold too low")
                return im_addr_gray
            if threshold > max_threshold:
                # print("Threshold too high")
                return im_addr_gray
            im_addr_binary= im_addr_gray.point( lambda p: 255 if p > threshold else 0 )
            hist = im_addr_binary.histogram()
            ratio = hist[0]/(hist[-1]+1)
            # print(f'New ratio: {ratio}')
        return im_addr_gray.point( lambda p: 255 if p > 200 else 0 )
    except ZeroDivisionError:
        print("B/(W+1) Ratio raised zero division error unexpectedly.")
        return im_addr_gray


def isolate_glyph(img, ind):
	'''Isolates one of 12 glyphs in a 768x64 addr image
	'''
	# Setting the points for cropped image
	left = 0 + ind*64
	right = left+64
	top = 0
	bottom = 64
	
	# Cropped image of above dimension
	# (It will not change original image)
	return img.crop((left, top, right, bottom))



def extract_glyphs(image_url, classifier):
    """Fetches an image url and extracts glpyhs, returning a 12-character hex string
    """

    # Default parameters
    null_address = "000000000000"       # location of central black hole 
    target_aspect_ratio = 16/9
    addr_size = (12*64, 64) 
    img_height = img_width = 64
    class_names = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","u"]
    note = ''


    # If Gallery, convert to image
    if image_url.split("/")[-2] == "gallery":
        image_url = gallery_first_image(image_url)
    
    if image_url == '': # no url, no address
        note='empty url'
        return null_address
    
    if image_url[-4:] != '.png' and image_url[-4:] != '.jpg': # no image, no address
        note = f'No Image at: {image_url}'
        return null_address
    
    # print(image_url)
    try: # Get image
        im = url_to_img(image_url)
    except: 
        note='Image error'
        return null_address
    im.save('temp_screenshot.png')
    
    width, height = im.size
    if abs((width / height) - target_aspect_ratio)>.01:
        note=f'Improper aspect ratio. {width}x{height}'
        return null_address       


    # Crop to just address portion (to avoid resizing everything to 4k)
    im_addr = img_to_address(im)
    im_addr.save('temp_address.png')
    
    # Preprocess Image
    
    # Resize address to be the size it is with 4k captures
    im_addr = im_addr.resize(size=addr_size, resample=Image.Resampling.LANCZOS)
    im_addr.save('temp_address_resize.png')

    # Convert color space to binary (ish)
    im_addr = color_to_binary(im_addr)
    im_addr.save('temp_address_binary.png')
    # return(im_addr)
    
    # Start empty address string
    address = ""

    # Loop through 12 glyphs
    # ! This was a terrible way to do this 
    # In the future, just store batches of images and classify a batch at a time
    for glyph_num in range(12):
        
        # Crop glyph
        glyph = isolate_glyph(im_addr, glyph_num)        
        
        # Save glyph temporarily
        temp_path = "glyph_to_classify.png"
        glyph.save(temp_path)
        
        # Classify Glyph
        tf_glyph =  tf.keras.utils.load_img(temp_path, target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(tf_glyph)
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        predictions = classifier.predict(img_array, verbose=0,)
        score = tf.nn.softmax(predictions[0])
        prediction = class_names[np.argmax(score)]

        # Append to String
        address += prediction

    # If any non-glyphs found, reset address
    if 'u' in address:
        address='000000000000'
        note='found non-glyphs'

    # return address
    return(address)

In [16]:
classifier = tf.keras.models.load_model('C:\\Users\\derek\\Dropbox\\Back to School\\Chico State\\Courses\\2022 Fall\\CSCI 490 Capstone\\capstone-nms-galactic-map\\production_scripts\\models\\glyphs_17')

In [239]:
index=0

In [263]:
index +=1
# url_to_img(df.url[index])
extract_glyphs(df.url[index], classifier)

https://i.redd.it/e57veebe49t61.png
temp.png


('41ACFDFFAFF8', '')

In [24]:
# df['note']='na'
for row in range(len(df)):
# for row in range(20):
    row_url = df.url[row]
    row_address = df['address'][row]
    if row % 10 == 0:
        print(f"---Row:{row}, Address:{row_address}")
    if row_address == "000000000000" or row_address == 0:
        if ".png" in row_url or '.jpg' in row_url: 
            # print("ping")
            try:
                row_address = extract_glyphs(row_url, classifier)
                if row_address != "000000000000" and row_address !=0:
                    print(f"{row}:{row_address}")
                    df['address'][row] = extract_glyphs(row_url, classifier)
            except:
                print(f"Error on Row: {row}. URL: {row_url}")

---Row:0, Address:0
---Row:10, Address:407903B93569
16:4191FD931FB3
---Row:20, Address:0
20:41ACFDFFAFF8
22:223301D41678
23:117200D3F676
---Row:30, Address:0
32:51B8FEFF4000
---Row:40, Address:0
40:3039046C21C2
41:514FFF30D0D8
---Row:50, Address:0
50:217AF7D43651
55:10320505890F
57:101010101101
59:6057FE56F3E5
---Row:60, Address:0
60:21EAFDF71EE1
Error on Row: 60. URL: https://i.redd.it/2osup98iw4y61.jpg
62:10069CF8092B
67:104A45EA9803
Error on Row: 68. URL: https://i.redd.it/z0e6n8cjkqx61.jpg
---Row:70, Address:0
71:31E8FFBBEC5A
72:3141FFBB2C48
---Row:80, Address:0
85:205CFED3263C
86:3089BCC7F654
89:40250D3FBE93
---Row:90, Address:0
90:515A0613D75C
94:100120200038
Error on Row: 95. URL: https://i.redd.it/wc2p6rr9fcw61.jpg
99:6106FE37AED8
---Row:100, Address:0
100:60798EAE626B
Error on Row: 105. URL: https://i.redd.it/4xc4zh7btjv61.jpg
109:213F25222222
---Row:110, Address:0
110:20F2849FD2E1
111:320406D44679
112:300F019FD325
Error on Row: 117. URL: https://i.redd.it/ia71mqjh2yu61.jpg
11

In [279]:
# df['address'] = df['url'].map(extract_glyphs())
# df['address'] = df.apply(lambda x: extract_glyphs(x['url'], classifier), axis=1)

# urls = list(df['url'])
# addresses = [extract_glyphs(url, classifier) for url in urls]
# addresses

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/conv2d_6/Relu' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\derek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\derek\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "C:\Users\derek\AppData\Local

In [25]:
df.groupby(['address']).address.count().sort_values(ascending=False)

address
0               6137
200110210111       9
420C0294C0C0       6
61E00294C0C0       6
3143F9DAD981       5
                ... 
1166C410DF52       1
1166043029E4       1
1166025C5265       1
1165FC376CEE       1
E1AFA549B52B       1
Name: address, Length: 2283, dtype: int64

In [63]:
# Suppress 'SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.'
pd.options.mode.chained_assignment = None  # default='warn'

from math import sqrt, floor

good_df = df[df['address']!=0]
good_df['body_index_h']     = good_df['address'].str[0]
good_df['sys_index_h']      = good_df['address'].str[1:4]
good_df['region_y_h']       = good_df['address'].str[4:6]
good_df['region_x_h']       = good_df['address'].str[6:9]
good_df['region_z_h']       = good_df['address'].str[9:12]

good_df['planet_index'] = good_df.body_index_h.apply(lambda x: int(x, 16))
good_df['sys_index']    = good_df.sys_index_h.apply(lambda x: int(x, 16))
good_df['y']            = good_df.region_y_h.apply(lambda x: int(x, 16) if int(x, 16) < 128 else int(x, 16)-255)
good_df['x']            = good_df.region_x_h.apply(lambda x: int(x, 16) if int(x, 16) < 2048 else int(x, 16)-4095)
good_df['z']            = good_df.region_z_h.apply(lambda x: int(x, 16) if int(x, 16) < 2048 else int(x, 16)-4095)
good_df['k_ly_from_center'] = good_df.apply(lambda x: floor(sqrt(x.x*x.x + x.y*x.y + x.z*x.z)*.4), axis=1)

,Unnamed: 0,post_id,author_id,flair_text,subject,galaxy,update,url,address,body_index_h,sys_index_h,region_y_h,region_z_h,region_x_h,planet_index,sys_index,y,x,z,k_ly_from_center
4,4,muj1up,85ogpn3s,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/achtdzj9f9u61.png,403800D1A668,4,038,00,668,D1A,4,56,0,-741,1640,719
6,6,mug4th,5khq221f,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/fdr9ncimi8u61.png,41ED01414591,4,1ED,01,591,414,4,493,1,1044,1425,706
10,10,mtwpcc,7hjepf8v,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/h5evcyaak3u61.jpg,407903B93569,4,079,03,569,B93,4,121,3,-1132,1385,715
16,16,ms04ep,6rch38,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/7e0z7bejzht61.jpg,4191FD931FB3,4,191,FD,FB3,931,4,401,-2,-1742,-76,697
20,20,mr62te,ldqbfm9,/3.35/Euclid/Planet,Planet,Euclid,3.35,https://i.redd.it/e57veebe49t61.png,41ACFDFFAFF8,4,1AC,FD,FF8,FFA,4,428,-2,-5,-7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8546,8546,w643sp,3vaqvag7,/3.94/Euclid/Frigate,Frigate,Euclid,3.94,https://i.redd.it/3b7vh5vglbd91.jpg,110D02FFF008,1,10D,02,008,FFF,1,269,2,0,8,3
8547,8547,w63xom,gz2wjd2w,/3.94/Euclid/Frigate,Frigate,Euclid,3.94,https://i.redd.it/7c4s4501kbd91.jpg,1111F7551C31,1,111,F7,C31,551,1,273,-8,1361,-974,669
8548,8548,w5zrje,tdcj2,/3.94/Euclid/Frigate,Frigate,Euclid,3.94,https://i.redd.it/l83w230icad91.png,100B0017D6DC,1,00B,00,6DC,17D,1,11,0,381,1756,718
8597,8597,yittp4,6791l,/4/Euclid/Frigate,Frigate,Euclid,4,https://i.redd.it/kth8ecjri8x91.jpg,5079FF996064,5,079,FF,064,996,5,121,0,-1641,100,657


In [85]:
good_df.drop(columns=["Unnamed: 0"], inplace=True)
good_df.dtypes

post_id             object
author_id           object
flair_text          object
subject             object
galaxy              object
update              object
url                 object
address             object
body_index_h        object
sys_index_h         object
region_y_h          object
region_z_h          object
region_x_h          object
planet_index         int64
sys_index            int64
y                    int64
x                    int64
z                    int64
k_ly_from_center     int64
dtype: object

In [86]:
good_df.to_csv('reddit_star_systems.csv')

In [91]:
# Distribution of distances from center
import plotly.io as pio
from plotly.figure_factory import create_distplot
pio.templates.default = "plotly_dark"


hist_data = [good_df.k_ly_from_center]
group_labels = ['ly to center/1000']
fig = create_distplot(hist_data, group_labels, bin_size=25)
fig.show()

In [98]:
list(good_df.subject.value_counts().values)

[754, 640, 368, 316, 94, 27]

Print custom output for use with convenient, nicely styled [online scatterplot tool](miabellaai.net).

In [84]:
import random

print("No Man's Sky Known Galaxy Map;")
print("::X::Z::Y;")

print("Galactic Center::0::0::0::-10::1::Landmark::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::Landmark::0::0::0::0::0;")
print("Galactic Center::0::0::0::-10::1::Inner Galaxy::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::Inner Galaxy::0::0::0::0::0;")
print("Galactic Center::0::0::0::-10::1::none::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::none::0::0::0::0::0;")
print("Galactic Center::0::0::0::-10::1::Core::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::Core::0::0::0::0::0;")
print("Galactic Center::0::0::0::-10::1::Glitter Band::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::Glitter Band::0::0::0::0::0;")
print("Galactic Center::0::0::0::-10::1::Outer Rim::0::0::0::0::0;")
print("Galactic Center::0::0::0::10::1::Outer Rim::0::0::0::0::0;")
print("Galactic Center::0::0::0::-3::10::Core::1::0::0::0::0;")
print("Accretion Disc::0::0::0::4::9::Core::.1::0::0::1::.4;")

x_range = z_range = [-2048,2048]
y_range = [-255,255]

# Create corner points to force range to desired size
for x in x_range:
    for y in y_range:
        for z in z_range:
            print(f"corner::{x}::{z}::{4*y}::-4::1::Landmark::0::0::0::1::1;")


# for row in range(len(good_df)):
for index, row in good_df.iterrows():
    y=row.y
    x=row.x
    z=row.z
    label=row.address
    jitter = 0
    blink_rate = 0
    size_change = 0
    stripe = 0
    transparency = 1
    color = random.uniform(-2.5, 2.5)
    size = random.uniform(1.5, 3.0)
    transparency = random.uniform(0, 1)
    if row.k_ly_from_center < 30:
        group = 'Core'
    elif row.k_ly_from_center < 675:
        group = 'Inner Galaxy'
    elif row.k_ly_from_center < 725:
        group = 'Glitter Band'
        size = random.uniform(1.0, 2.0)
        blink_rate = random.uniform(0, 1)
    else:
        group = 'Outer Rim'
    print(f'{label}::{x}::{z}::{y}::{color}::{size}::{group}::{transparency}::{blink_rate}::{jitter}::{stripe}::{size_change};')


No Man's Sky Known Galaxy Map;
::X::Z::Y;
Galactic Center::0::0::0::-10::1::Landmark::0::0::0::0::0;
Galactic Center::0::0::0::10::1::Landmark::0::0::0::0::0;
Galactic Center::0::0::0::-10::1::Inner Galaxy::0::0::0::0::0;
Galactic Center::0::0::0::10::1::Inner Galaxy::0::0::0::0::0;
Galactic Center::0::0::0::-10::1::none::0::0::0::0::0;
Galactic Center::0::0::0::10::1::none::0::0::0::0::0;
Galactic Center::0::0::0::-10::1::Core::0::0::0::0::0;
Galactic Center::0::0::0::10::1::Core::0::0::0::0::0;
Galactic Center::0::0::0::-10::1::Glitter Band::0::0::0::0::0;
Galactic Center::0::0::0::10::1::Glitter Band::0::0::0::0::0;
Galactic Center::0::0::0::-10::1::Outer Rim::0::0::0::0::0;
Galactic Center::0::0::0::10::1::Outer Rim::0::0::0::0::0;
Galactic Center::0::0::0::-3::10::Core::1::0::0::0::0;
Accretion Disc::0::0::0::4::9::Core::.1::0::0::1::.4;
corner::-2048::-2048::-1020::-4::1::Landmark::0::0::0::1::1;
corner::-2048::2048::-1020::-4::1::Landmark::0::0::0::1::1;
corner::-2048::-2048::10

In [52]:
print("Color test points;")
print("::X::Y::Z;")

y=0
size = 10
group = "_"
transparency = 1
blink_rate = 0
jitter = 0
stripe = 0
size_change = 0

for group in range(0,6):
    for x in range(-10,10):
        z=100*group
        color = x
        print(f'Color={color}::{100*x}::{y}::{z}::{color}::{size}::{group}::{transparency}::{blink_rate}::{jitter}::{stripe}::{size_change};')

Color test points;
::X::Y::Z;
Color=-10::-1000::0::0::-10::10::0::1::0::0::0::0;
Color=-9::-900::0::0::-9::10::0::1::0::0::0::0;
Color=-8::-800::0::0::-8::10::0::1::0::0::0::0;
Color=-7::-700::0::0::-7::10::0::1::0::0::0::0;
Color=-6::-600::0::0::-6::10::0::1::0::0::0::0;
Color=-5::-500::0::0::-5::10::0::1::0::0::0::0;
Color=-4::-400::0::0::-4::10::0::1::0::0::0::0;
Color=-3::-300::0::0::-3::10::0::1::0::0::0::0;
Color=-2::-200::0::0::-2::10::0::1::0::0::0::0;
Color=-1::-100::0::0::-1::10::0::1::0::0::0::0;
Color=0::0::0::0::0::10::0::1::0::0::0::0;
Color=1::100::0::0::1::10::0::1::0::0::0::0;
Color=2::200::0::0::2::10::0::1::0::0::0::0;
Color=3::300::0::0::3::10::0::1::0::0::0::0;
Color=4::400::0::0::4::10::0::1::0::0::0::0;
Color=5::500::0::0::5::10::0::1::0::0::0::0;
Color=6::600::0::0::6::10::0::1::0::0::0::0;
Color=7::700::0::0::7::10::0::1::0::0::0::0;
Color=8::800::0::0::8::10::0::1::0::0::0::0;
Color=9::900::0::0::9::10::0::1::0::0::0::0;
Color=-10::-1000::0::100::-10::10::1::1

Custom points MiaBellaAI example:
```
Interactive 3D Scatter Plot;
::X::Y::Z;
Point A 1::65::42.78::7::19465.17::13::A::0::0::0::1::0;
Point A 2::73.18::44.61::9.72::31729.42::13::A::0::0::0::1::0;
Point A 3::80.46::44.94::12.44::44976.74::13::A::0::0::0::1::0;
Point A 4::86.04::43.73::15.16::57038.02::13::A::0::0::0::1::0;
Point A 5::89.3::41.12::17.88::65659.62::13::A::0::0::0::1::0;
Point A 6::89.89::37.4::20.6::69254.52::13::A::0::0::0::1::0;
Point A 7::87.73::32.98::23.32::67465.62::13::A::0::0::0::1::0;
Point A 8::83.08::28.33::26.04::61291.54::13::A::0::0::0::1::0;
Point A 9::76.43::23.98::28.76::52717::13::A::0::0::0::1::0;
Point A 10::68.53::20.4::31.48::44018.41::13::A::0::0::0::1::0;
Point A 11::60.24::17.99::34.2::37068.38::13::A::0::0::0::1::0;
Point A 12::52.47::17.01::36.92::32959.4::13::A::0::0::0::1::0;
Point A 13::46.08::17.58::39.64::32102.76::13::A::0::0::0::1::0;
Point A 14::41.77::19.61::42.36::34707.34::13::A::0.01::0::0::1::0;
Point A 15::40.03::22.91::45.08::41329.68::13::A::0.01::0::0::1::0;
Point A 16::41.03::27.09::47.8::53122.24::13::A::0.01::0::0::1::0;
Point A 17::44.67::31.7::50.52::71537.12::13::A::0.01::0::0::1::0;
Point A 18::50.55::36.24::53.24::97517.43::13::A::0.02::0::0::1::0;
Point A 19::58.01::40.2::55.96::130502.11::13::A::0.02::0::0::1::0;
Point A 20::66.25::43.14::58.68::167736.21::13::A::0.03::0::0::1::0;
Point A 21::74.35::44.76::61.4::204321.49::13::A::0.03::0::0::1::0;
Point A 22::81.42::44.85::64.12::234164.89::13::A::0.04::0::0::1::0;
Point A 23::86.69::43.42::66.84::251595.88::13::A::0.04::0::0::1::0;
Point A 24::89.56::40.63::69.56::253097.45::13::A::0.05::0::0::1::0;
Point A 25::89.73::36.77::72.28::238486.92::13::A::0.06::0::0::1::0;
Point A 26::87.18::32.28::75::211058.28::13::A::0.07::0::0::1::0;
Point A 27::82.19::27.65::77.72::176597.79::13::A::0.09::0::0::1::0;
Point A 28::75.3::23.38::80.44::141643.76::13::A::0.1::0::0::1::0;
Point A 29::67.28::19.96::83.16::111677.65::13::A::0.11::0::0::1::0;
Point A 30::59.01::17.75::85.88::89957.81::13::A::0.13::0::0::1::0;
Point A 31::51.4::17::88.6::77418.51::13::A::0.15::0::0::1::0;
Point A 32::45.28::17.79::91.32::73572.73::13::A::0.17::0::0::1::0;
Point A 33::41.34::20.04::94.04::77892.43::13::A::0.19::0::0::1::0;
Point A 34::40::23.49::96.76::90908.43::13::A::0.21::0::0::1::0;
Point A 35::41.41::27.77::99.48::114391.87::13::A::0.24::0::0::1::0;
Point A 36::45.42::32.4::102.2::150411.39::13::A::0.27::0::0::1::0;
Point A 37::51.59::36.88::104.92::199620.83::13::A::0.3::0::0::1::0;
Point A 38::59.23::40.72::107.64::259566.58::13::A::0.33::0::0::1::0;
Point A 39::67.5::43.48::110.36::323901.85::13::A::0.37::0::0::1::0;
Point A 40::75.5::44.87::113.08::383087.83::13::A::0.41::0::0::1::0;
Point A 41::82.35::44.73::115.8::426559.06::13::A::0.45::0::0::1::0;
Point A 42::87.28::43.08::118.52::445685.29::13::A::0.5::0::0::1::0;
Point A 43::89.77::40.1::121.24::436457.41::13::A::0.55::0::0::1::0;
Point A 44::89.52::36.12::123.96::400855.58::13::A::0.6::0::0::1::0;
Point A 45::86.58::31.58::126.68::346331.92::13::A::0.66::0::0::1::0;
Point A 46::81.26::26.97::129.4::283574.15::13::A::0.72::0::0::1::0;
Point A 47::74.15::22.8::132.12::223402.77::13::A::0.78::0::0::1::0;
Point A 48::66.03::19.54::134.84::173995.17::13::A::0.85::0::0::1::0;
Point A 49::57.8::17.54::137.56::139469.27::13::A::0.92::0::0::1::0;
Point A 50::50.37::17.02::140.28::120255.86::13::A::1::0::0::1::0;
Point B 1::2.02::33.58::109.75::109.75::8.01::B::1::0::0::0::0;
Point B 2::-8.19::27.56::109.4::109.4::12.18::B::1::0::0::0::0;
Point B 3::-30.58::30.44::129.69::129.69::16.66::B::1::0::0::0::0;
Point B 4::-31.94::36.16::135.09::135.09::13.89::B::1::0::0::0::0;
Point B 5::-42.95::26.67::118.21::118.21::12.92::B::1::0::0::0::0;
Point B 6::-46.46::28.86::87.89::87.89::3.15::B::1::0::0::0::0;
Point B 7::-52.7::33.88::125.84::125.84::8.3::B::1::0::0::0::0;
Point B 8::-32.93::45.45::125.22::125.22::14.35::B::1::0::0::0::0;
Point B 9::-5.75::25.58::115.19::115.19::11.91::B::1::0::0::0::0;
Point B 10::-33.26::30.66::113.84::113.84::17.11::B::1::0::0::0::0;
Point B 11::-24.72::34.65::121.39::121.39::21::B::1::0::1::0::0;
Point B 12::-13.96::36.29::109.9::109.9::14.66::B::1::0::0::0::0;
Point B 13::-2.58::32.84::129.07::129.07::10.35::B::1::0::0::0::0;
Point B 14::-17.68::37.14::103.23::103.23::12.96::B::1::0::0::0::0;
Point B 15::6.16::34.54::124.15::124.15::6.74::B::1::0::0::0::0;
Point B 16::-22.24::26.49::126.28::126.28::17.78::B::1::0::0::0::0;
Point B 17::-1.39::45.45::118.34::118.34::8.97::B::1::0::0::0::0;
Point B 18::-20.98::26.96::122.35::122.35::18.7::B::1::0::0::0::0;
Point B 19::-21.03::38.05::143.18::143.18::10.36::B::1::0::0::0::0;
Point B 20::-28.99::37.95::108.72::108.72::15.79::B::1::0::0::0::0;
Point B 21::-30.05::32.86::116.03::116.03::19.1::B::1::0::0.05::0::0;
Point B 22::-23.45::27.81::126.73::126.73::18.15::B::1::0::0::0::0;
Point B 23::-32.58::38.21::124.82::124.82::17.02::B::1::0::0::0::0;
Point B 24::-19.6::23.48::124.77::124.77::16.58::B::1::0::0::0::0;
Point B 25::0.14::39.83::120.39::120.39::9.36::B::1::0::0::0::0;
Point B 26::-16.92::34.06::108.1::108.1::15.14::B::1::0::0::0::0;
Point B 27::9.67::26.93::119.14::119.14::4.96::B::1::0::0::0::0;
Point B 28::-27.79::27.56::105.35::105.35::14.52::B::1::0::0::0::0;
Point B 29::-38.87::44.19::106.13::106.13::11.01::B::1::0::0::0::0;
Point B 30::-34.06::45.45::117.33::117.33::14.37::B::1::0::0::0::0;
Point B 31::-26.99::27.63::114.77::114.77::18.61::B::1::0::0::0::0;
Point B 32::-28.98::30.53::143.18::143.18::10.63::B::1::0::0::0::0;
Point B 33::-36.1::33.75::107.16::107.16::13.88::B::1::0::0::0::0;
Point B 34::-26.61::26.37::131.44::131.44::15.82::B::1::0::0::0::0;
Point B 35::-46.34::32.09::110.76::110.76::10.8::B::1::0::0::0::0;
Point B 36::-46.14::32.32::130.17::130.17::10.71::B::1::0::0::0::0;
Point B 37::-34.14::43.6::102.48::102.48::11.1::B::1::0::0::0::0;
Point B 38::-23.94::33.88::97.08::97.08::10.94::B::1::0::0::0::0;
Point B 39::-38.31::27.64::123.65::123.65::15.06::B::1::0::0::0::0;
Point B 40::-21.06::38.37::106.58::106.58::14.79::B::1::0::0::0::0;
Point B 41::-23.67::34.15::107.17::107.17::15.86::B::1::0::0::0::0;
Point B 42::-51.44::30.02::121.71::121.71::9.15::B::1::0::0::0::0;
Point B 43::-38.38::37.76::111.24::111.24::13.97::B::1::0::0::0::0;
Point B 44::-40.95::40.73::115.96::115.96::13.2::B::1::0::0::0::0;
Point B 45::-43.31::27.94::109.24::109.24::11.56::B::1::0::0::0::0;
Point B 46::-28.39::33.91::124.96::124.96::19.23::B::1::0::0.11::0::0;
Point B 47::-21.72::29.73::105.96::105.96::15.03::B::1::0::0::0::0;
Point B 48::-28.79::38.23::109.18::109.18::15.95::B::1::0::0::0::0;
Point B 49::-29.2::40.69::118.73::115.73::20.52::B::1::0::0.30::0::0;
Point B 50::-67.15::34.81::128.96::128.96::1::B::1::0::0::0::0;
Point C 1::64.43::-7.33::6.82::-0.51::50::C::1::0::0::0::0;
Point C 2::55.56::-9.45::8.88::-0.57::49::C::1::0::0::0::0;
Point C 3::47.11::-31.19::10.95::-20.24::48::C::1::0::0::0::0;
Point C 4::86.63::-14.93::13.01::-1.92::47::C::1::0::0::0::0;
Galaxy Center::0::0::0::-53.38::45::Landmark::1::0::0::0::0;
Galaxy Center::0::0::0::383087::50::Landmark::.5::0::0::1::.3;
Point C 6::39.47::-17.84::17.14::-0.7::45::C::1::0::0::0::0;
Point C 7::56.03::-20.66::19.2::-1.46::44::C::1::0::0::0::0;
Point C 8::48.24::-33.39::21.26::-12.13::43::C::1::0::0::0::0;
Point C 9::42.58::-13.39::23.33::9.93::42::C::1::0::0::0::0;
Point C 10::30.95::-22.81::25.39::2.58::41::C::1::0::0::0::0;
Point C 11::-56.2::-23.05::27.45::4.4::40::C::1::0::0::0::0;
Point C 12::-47.81::-22.54::29.52::6.98::39::C::1::0::0::0::0;
Point C 13::-75.05::-6.03::31.58::25.55::38::C::1::0::0::0::0;
Point C 14::-37.51::-11.5::33.65::22.15::37::C::1::0::0::0::0;
Point C 15::-61.49::-0.12::35.71::35.59::36::C::1::0::0::0::0;
Point C 16::-52.71::-7.62::37.77::30.15::35::C::1::0::0::0::0;
Point C 17::-57.95::-11.63::39.84::28.21::34::C::1::0::0::0::0;
Point C 18::-58.51::-26.02::41.9::15.88::33::C::1::0::0::0::0;
Point C 19::-58.97::-26.55::43.96::17.41::32::C::1::0::0::0::0;
Point C 20::-51.49::-3.68::46.03::42.34::31::C::1::0::0::0::0;
Point C 21::-62.35::-30.12::48.09::17.97::30::C::1::0::0::0::0;
Point C 22::-58.73::-11.23::50.15::38.92::29::C::1::0::0::0::0;
Point C 23::-3.11::-19.71::-52.22::32.51::28::C::1::0::0::0::0;
Point C 24::87.9::-32.34::54.28::21.95::27::C::1::0::0::0::0;
Point C 25::35.63::-24.63::56.35::31.71::26::C::1::0::0::0::0;
Point C 26::66.67::-23.17::58.41::35.24::25::C::1::0::0::0::0;
Point C 27::59::-22.35::60.47::38.13::24::C::1::0::0::0::0;
Point C 28::66.7::-28.88::62.54::33.66::23::C::1::0::0::0::0;
Point C 29::3.43::-7.67::64.6::56.93::22::C::1::0::0::0::0;
Point C 30::44.96::-25.7::66.66::40.96::21::C::1::0::0::0::0;
Point C 31::79.82::-23.64::68.73::45.09::20::C::1::0::0::0::0;
Point C 32::63.53::-9.34::70.79::61.45::19::C::1::0::0::0::0;
Point C 33::64.11::-16.99::72.85::55.86::18::C::1::0::0::0::0;
Point C 34::79.66::-15.85::74.92::59.07::17::C::1::0::0::0::0;
Point C 35::52.43::-16.2::76.98::60.79::16::C::1::0::0::0::0;
Point C 36::68.55::-34.93::79.05::44.12::15::C::1::0::0::0::0;
Point C 37::48.2::-20.81::81.11::60.29::14::C::1::0::0::0::0;
Point C 38::43.83::-0.36::83.17::82.82::13::C::1::0::0::0::0;
Point C 39::51.96::-2.91::85.24::82.33::12::C::1::0::0::0::0;
Point C 40::77.34::-29.44::87.3::57.86::11::C::1::0::0::0::0;
Point C 41::59.6::-31.02::89.36::58.34::10::C::1::0::0::0::0;
Point C 42::72.74::-25.76::91.43::65.66::9::C::1::0::0::0::0;
Point C 43::47.25::-14.8::93.49::78.69::8::C::1::0::0::0::0;
Point C 44::56.19::-20.69::95.55::74.87::7::C::1::0::0::0::0;
Point C 45::81.34::-19.28::97.62::78.33::6::C::1::0::0::0::0;
Point C 46::46.61::-16.04::99.68::83.64::5::C::1::0::0::0::0;
Point C 47::57.62::5.88::-101.75::107.63::4::C::1::0::0::0::0;
Point C 48::72.41::-24.44::103.81::79.37::3::C::1::0::0::0::0;
Point C 49::57.06::-26.78::105.87::79.09::2::C::1::0::0::0::0;
Point C 50::59.05::-10.26::107.94::97.68::1::C::1::0::0::0::0;
Point D 1::6.16::-15.44::6.82::6.16::1::D::1::0::0::0::0.02;
Point D 2::-2.96::-16.02::6.82::-2.96::1.7::D::1::0::0::0::0.04;
Point D 3::-1.71::-28.76::6.82::-1.71::2.4::D::1::0::0::0::0.06;
Point D 4::0.92::-35.99::6.82::0.92::3.1::D::1::0::0::0::0.08;
Point D 5::-3.52::-31.53::6.82::-3.52::3.8::D::1::0::0::0::0.1;
Point D 6::-6.7::-43.07::6.82::-6.7::4.5::D::1::0::0::0::0.12;
Point D 7::-6.49::-31.36::6.83::-6.49::5.2::D::1::0::0::0::0.14;
Point D 8::-5.35::-16.36::6.83::-5.35::5.9::D::1::0::0::0::0.16;
Point D 9::-3.78::-27.08::6.84::-3.78::6.6::D::1::0::0::0::0.18;
Point D 10::4.84::-22.92::6.86::4.84::7.3::D::1::0::0::0::0.2;
Point D 11::-6.37::-41.31::6.89::-6.37::8::D::1::0::0::0::0.22;
Point D 12::-15.92::-37.55::6.93::-15.92::8.7::D::1::0::0::0::0.24;
Point D 13::-7.32::-27.63::6.98::-7.32::9.4::D::1::0::0::0::0.26;
Point D 14::-2.79::-18.25::7.05::-2.79::10.1::D::1::0::0::0::0.28;
Point D 15::-12.06::-45.45::7.15::-12.06::10.8::D::1::0::0::0::0.3;
Point D 16::-0.78::-31.7::7.28::-0.78::11.5::D::1::0::0::0::0.32;
Point D 17::-5.62::-27.96::7.44::-5.62::12.2::D::1::0::0::0::0.34;
Point D 18::-12.22::-15.06::7.64::-12.22::12.9::D::1::0::0::0::0.36;
Point D 19::-8.65::-33.24::7.9::-8.65::13.6::D::1::0::0::0::0.38;
Point D 20::-11.42::-37.14::8.21::-11.42::14.3::D::1::0::0::0::0.4;
Point D 21::-16.68::-32.04::8.6::-16.68::15::D::1::0::0::0::0.42;
Point D 22::-16.63::-31.27::9.07::-16.63::15.7::D::1::0::0::0::0.44;
Point D 23::-18.43::-27.11::9.63::-18.43::16.4::D::1::0::0::0::0.46;
Point D 24::-20.25::-42.09::10.29::-20.25::17.1::D::1::0::0::0::0.48;
Point D 25::-17.6::-16.87::11.08::-17.6::17.8::D::1::0::0::0::0.5;
Point D 26::-21.23::-34.68::12::-21.23::18.5::D::1::0::0::0::0.52;
Point D 27::-29.19::-29.06::13.08::-29.19::19.2::D::1::0::0::0::0.54;
Point D 28::-29.99::-28.86::14.33::-29.99::19.9::D::1::0::0::0::0.56;
Point D 29::-23.05::-18.8::15.77::-23.05::20.6::D::1::0::0::0::0.58;
Point D 30::-30.55::0.33::17.42::-30.55::21.3::D::1::0::0::0::0.6;
Point D 31::-35.25::-37.52::19.31::-35.25::22::D::1::0::0::0::0.62;
Point D 32::-36.9::-22.38::21.46::-36.9::22.7::D::1::0::0::0::0.64;
Point D 33::-43.68::-15.16::23.9::-43.68::23.4::D::1::0::0::0::0.66;
Point D 34::-45.91::-40.85::26.64::-45.91::24.1::D::1::0::0::0::0.68;
Point D 35::-46.39::-30.64::29.74::-46.39::24.8::D::1::0::0::0::0.7;
Point D 36::-53.38::-37.31::33.2::-53.38::25.5::D::1::0::0::0::0.72;
Point D 37::-47.33::-1.51::37.08::-47.33::26.2::D::1::0::0::0::0.74;
Point D 38::-47.66::-37.75::41.39::-47.66::26.9::D::1::0::0::0::0.76;
Point D 39::-43.76::-16.77::46.19::-43.76::27.6::D::1::0::0::0::0.78;
Point D 40::-51.79::-8.59::51.5::-51.79::28.3::D::1::0::0::0::0.8;
Point D 41::-53.04::-5.74::57.37::-53.04::29::D::1::0::0::0::0.82;
Point D 42::-69.25::-28.94::63.85::-69.25::29.7::D::1::0::0::0::0.84;
Point D 43::-61.35::-27::70.97::-61.35::30.4::D::1::0::0::0::0.86;
Point D 44::-70.47::-8.07::78.78::-70.47::31.1::D::1::0::0::0::0.88;
Point D 45::-73.14::-11.16::87.34::-73.14::31.8::D::1::0::0::0::0.9;
Point D 46::-77.72::-14.23::96.69::-77.72::32.5::D::1::0::0::0::0.92;
Point D 47::-81.89::-4.25::106.9::-81.89::33.2::D::1::0::0::0::0.94;
Point D 48::-79.09::-11.37::118.01::-79.09::33.9::D::1::0::0::0::0.96;
Point D 49::-90.91::-1.96::130.08::-90.91::34.6::D::1::0::0::0::0.98;
Point D 50::-90.91::-6.98::143.18::-90.91::35.3::D::1::0::0::0::1;
```